In [3]:
import os
import zipfile
import json

In [4]:
%cd ..

C:\Users\Caitlin\Documents\Brett_TTT_projects\facial_age_estimation_cnn


C:\Users\Caitlin\Anaconda3\envs\sprint_15_env\lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# Define path to zipped TensorBoard logs
zip_path = os.path.join('logs', 'tensorboard_logs.zip')
extracted_log_dir = os.path.join('logs', 'tensorboard_logs')

In [4]:
# Unzip logs only if not already extracted
if not os.path.exists(extracted_log_dir):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_log_dir)
    print(f"Extracted logs to: {extracted_log_dir}")
else:
    print(f"Logs already extracted at: {extracted_log_dir}")

Logs already extracted at: logs\tensorboard_logs


In [7]:
%load_ext tensorboard
%tensorboard --logdir logs/tensorboard_logs --port 6007

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Our tensorboard output shows that our loss function (MSE) improvement plateaus before our evaluation metric (MAE) plateaus. In fact, we seem to still be getting steady (though small) imporvement in MAE during our final epoch. This effect can be seen most prominently on the evaluation_mae_vs_iterations graph that can be found in the SCALARS tab. This suggests that our model may benefit from a few more epochs of training, even though improvement to the loss function appears to have plateaued. One consideration which might be tempting would be to switch to MAE loss later in training since MSE has topped out. Hoewver, because our model's average error is still greater than 1, this is likely to shrink the size of our gradients, leading to a vanishing gradient problem that would take place sooner than it would if we continued with MSE as our loss function. Ironically, then, switching to an MAE loss function might actually decrease model improvement as measured by MAE. 

The consistent disparity between performance on the training set and the validation set suggests that our model might benefit from some added regularization. However, it is worth noting that we saw increasing convergence of range of weights throughout training suggesting that our model is already regularizing pretty well (this can be seen in the HISTOGRAM tab under kernel/histogram). It is also worth noting that a couple iterations of this network were run with dropout and did not show noticeable improvement. 

Another potential solution that may prolong network learning would be to replace the ReLu neurons in ResNet50 with leaky ReLu neurons. While ResNet50 already utilizes skip residual connections to prevent a dying ReLu problem, when and where there are negative inputs to a ReLu neuron skip residual connections would do nothing to prevent a dying ReLu problem. If what is fed into a neuron of a subsequent layer is F(x) + x and x is negative, ReLu neurons entail that F(x) = 0 and x < 0. Therefore, swapping leaky ReLu neurons for standard ReLu neruons offers significant potential in prolonging learning and improving learning quality. If we really wanted to check the gradients with respect to our model's parameters before implementing such a strategy we could initiate a gradient logger class and utilize the get_gradients() method from keras to see if our model exhibits a dying ReLu problem at all during training. 

In [5]:
def display_results(results_name: str) -> None:
    # Path to your JSON file
    json_path = os.path.join('metrics', results_name)
    # Load JSON data
    with open(json_path, 'r') as f:
        eval_results = json.load(f)
    # Display results
    for key, value in eval_results.items():
        print(f"{key}: {value}")

In [6]:
display_results("model_eval_results.json")

model_mae: 8.619847297668457
model_mse: 144.00405883789062
human_mae: 4.58948056502737
human_mse: 41.15976503909733


At present our model is underperforming relative to the human baseline MAE of 4.59 by about 4 years. Given the much larger disparity on MSE, this may be due to a few large errors rather than consistent inaccuracy. After all, our model was fed a range of ages from 1 to 100 with significantly fewer training examples for ages over 60 (see EDA.ipynb in the notebooks subdirectory). Therefore, it remains possible that our model equals, or potentially even exceeds, human performance on the relevant age ranges (roughly 13 to 40, or ages where appearnce may not always gauruntee age > or < 21). To check this we can restrict the evaluation to the model's performance on ages >= 13 and <= 40. See the below output. 

In [11]:
display_results("model_eval_relevant.json")

model_mae: 5.52546501159668
model_mse: 48.299922943115234
human_mae: 4.2348189641046785
human_mse: 29.276154085785556


This is fairly remarkable improvement. For what we might consider to be "close calls" the model performs with near human leveal accuracy, with the average error by the model being only 1.29 greater than that of the human baseline. Replacing a human cashier with this level of performance would not be unreasonable, insofar as the model can predict the right side of 21 for non "close calls". We will check that in the next couple of cells.  

In [12]:
display_results("model_eval_13.json")

Condition: Predictions >= 21 for real_age <= 13
Num_samples: 274
Num_wrong_model: 40
Percent_wrong_model: 14.5985401459854
Num_wrong_humans: 7
Percent_wrong_humans: 2.5547445255474455


Unfortunately, the model's error rate is unacceptable error rate for customers 13 or younger, and it is well below the human baseline. with this kind of performance, this model is not quite ready to replace Good Seed's cashier for them. 

In [13]:
display_results("model_eval_40.json")

Condition: Predictions < 21 for real_age >= 40
Num_samples: 620
Num_wrong_model: 11
Percent_wrong_model: 1.7741935483870968
Num_wrong_humans: 1
Percent_wrong_humans: 0.16129032258064516


The model performed much better here. Moreover, errors on this side of 21 are much less consequential. Depending on the technology available to good seed, we could imagine them installing a "challenge" option for customers who believe the machine is denying them unjustly. When such a button is clicked whoever is in back could come out to check the individual out (thereby bypassing the need for a cashier). Underage buyers would be extremely unlikely to make such a move, and even if they did, they would be unlikely to make it past the real person who came out to finish checking the customer out. Moreover, there would be no legal liability associated with denying an older customer service, although the consequences of poor performance on this side of 21 would likely negatively impact Good Seed's popularity. 

## Summary and Conclusions

The fact that the model performed much better on "close calls" than it did on jdugements that should be easier to make like whether or not a 13 year old is 21, suggests that the biggest problem with the model is not the model itself, but the training data available to the model. Simply put, if the model had access to more training data for customers between the ages of 13 and 21 and for customers 40 and older, it would likely be capable of approving the purchase of alcohol without oversight from humans.

That said, we know that the CNN discussed in the paper where the APPA REAL Dataset was first used acheived an MAE of 5.29 so we know performacne can be improved even with this same dataset. See [here](https://chalearnlap.cvc.uab.cat/dataset/26/description/). Considering the structure of that CNN would, obviously, provide some useful ideas for improving the performance of the CNN here. That paper is titled "Apparent and Real Age Estimation in Still Images with Deep Residual Regressors on APPA-REAL Database" by Agustsson et al., published in 2017, and can be accessed [here](https://inria.hal.science/hal-01677892/file/apparent_age_FG2017.pdf). One notable difference between Agustsson et al.'s model and the model constructed here is that their model trained on residuals (Agustsson et al. pg 5).  

Nevertheless, we noted some areas where the model could potentially be improved on our own. In the future we can try adding a few extra epochs to allow the model to continue to improve its performance, experimenting with different regularization techniques, and replacing the ReLu activation functions in ResNet50 with leaky ReLu functions.